In [1]:
import numpy as np
import cv2
import time

In [17]:
driveName = r"\\.\PhysicalDrive1"  # Change this to the correct drive

# SD Card sector information
headerSector =          1023 # Holds user settings to configure Miniscope and recording
configSector =          1024 # Holds final settings of the actual recording
dataStartSector =       1025 # Recording data starts here
sectorSize =            512

WRITE_KEY0 =				0x0D7CBA17
WRITE_KEY1 =				0x0D7CBA17
WRITE_KEY2 =				0x0D7CBA17
WRITE_KEY3 =				0x0D7CBA17

# SD Card Header Sector positions
HEADER_GAIN_POS =				4
HEADER_LED_POS =				5
HEADER_EWL_POS =				6
HEADER_RECORD_LENGTH_POS =  	7
HEADER_FRAME_RATE = 			8

# SD Card Config Sector positions
CONFIG_BLOCK_WIDTH_POS =			0
CONFIG_BLOCK_HEIGHT_POS	=   		1
CONFIG_BLOCK_FRAME_RATE_POS	=   	2
CONFIG_BLOCK_BUFFER_SIZE_POS =  	3

# Data Buffer Header positions
BUFFER_HEADER_HEADER_LENGTH_POS =			0
BUFFER_HEADER_LINKED_LIST_POS = 			1
BUFFER_HEADER_FRAME_NUM_POS	=   			2
BUFFER_HEADER_BUFFER_COUNT_POS =			3
BUFFER_HEADER_FRAME_BUFFER_COUNT_POS =  	4
BUFFER_HEADER_TIMESTAMP_POS	=   			5
BUFFER_HEADER_DATA_LENGTH_POS = 			6

In [6]:
# variables that you can adjust
correctDrive = False
f = open(driveName, "rb")  # Open drive

# Make sure this is the correct drive
# Read SD Card header and config sectors
f.seek(headerSector * sectorSize, 0)  # Move to correct sector
headerSectorData = np.fromstring(f.read(sectorSize), dtype=np.uint32)
if ((WRITE_KEY0 == headerSectorData[0]) and (WRITE_KEY1 == headerSectorData[1]) and (WRITE_KEY2 == headerSectorData[2]) and (WRITE_KEY3 == headerSectorData[3])):
    correctDrive = True
else:
    print ("Wrong Drive")
    correctDrive = False
    f.close()

Wrong Drive


In [7]:
# Load up Config Sector
f.seek(configSector * sectorSize, 0)  # Move to correct sector
configSectorData = np.fromstring(f.read(sectorSize), dtype=np.uint32)

In [24]:
# Read Data Sectors
frameNum = -1
pixelCount = 0
frameBufferCount = 0
frame = np.empty((configSectorData[CONFIG_BLOCK_WIDTH_POS] * configSectorData[CONFIG_BLOCK_HEIGHT_POS], 1), dtype=np.uint8)
f.seek(dataStartSector * sectorSize, 0) # Starting data location
for i in range(1000):
    dataHeader = np.fromstring(f.read(4), dtype=np.uint32)
    dataHeader = np.append(dataHeader, np.fromstring(f.read((dataHeader[BUFFER_HEADER_HEADER_LENGTH_POS] - 1) * 4), dtype=np.uint32))
    print(dataHeader)
    data = np.fromstring(f.read((dataHeader[BUFFER_HEADER_DATA_LENGTH_POS])), dtype=np.uint8)
    print(data.shape)
    # print(i)
    
    # -------------------------------------
    # Everything below can be done much better!
    if (dataHeader[BUFFER_HEADER_FRAME_NUM_POS] == frameNum):
        if (frameBufferCount == 5):
            remaining = frame.size - pixelCount
            frame[pixelCount:, 0] = data[:remaining]
            cv2.imshow('Video', np.reshape(frame, (configSectorData[CONFIG_BLOCK_WIDTH_POS], configSectorData[CONFIG_BLOCK_HEIGHT_POS] )))
            cv2.waitKey(50)
            # time.sleep(.2)
        else:
            frame[pixelCount:(pixelCount + dataHeader[BUFFER_HEADER_DATA_LENGTH_POS]), 0] = data

        pixelCount = pixelCount + dataHeader[BUFFER_HEADER_DATA_LENGTH_POS]
        frameBufferCount = frameBufferCount + 1
        # print(frameBufferCount)
    else:
        # First buffer of frame
        frame[0:dataHeader[BUFFER_HEADER_DATA_LENGTH_POS], 0] = data

        pixelCount = dataHeader[BUFFER_HEADER_DATA_LENGTH_POS]
        frameBufferCount = 1
        frameNum = frameNum + 1
        
cv2.destroyWindow('Video')


86 63972]
(63972,)
[    7     3   103   623     5  5290 63972]
(63972,)
[    7     4   104   624     0  5320 63972]
(63972,)
[    7     0   104   625     1  5324 63972]
(63972,)
[    7     1   104   626     2  5328 63972]
(63972,)
[    7     2   104   627     3  5332 63972]
(63972,)
[    7     3   104   628     4  5337 63972]
(63972,)
[    7     4   104   629     5  5340 63972]
(63972,)
[    7     0   105   630     0  5370 63972]
(63972,)
[    7     1   105   631     1  5374 63972]
(63972,)
[    7     2   105   632     2  5379 63972]
(63972,)
[    7     3   105   633     3  5383 63972]
(63972,)
[    7     4   105   634     4  5387 63972]
(63972,)
[    7     0   105   635     5  5391 63972]
(63972,)
[    7     1   106   636     0  5421 63972]
(63972,)
[    7     2   106   637     1  5425 63972]
(63972,)
[    7     3   106   638     2  5429 63972]
(63972,)
[    7     4   106   639     3  5434 63972]
(63972,)
[    7     0   106   640     4  5438 63972]
(63972,)
[    7     1   106   641   